In [ ]:
import subprocess
import os
from datetime import datetime

decoders = ["uf", "clayg"]
distances = [4,6]
start = 10**-2
end = 3*10**-2
step = ((10**-4/10*-2)**(1/10))*0.99

cwd = os.getcwd()
base_dir = os.path.join(cwd, "data/treshold_plots")
os.makedirs(base_dir, exist_ok=True)

existing_ids = [
    int(f.split("-")[0]) for f in os.listdir(base_dir) 
    if os.path.isdir(os.path.join(base_dir, f)) and f.split("-")[0].isdigit()
]

next_id = max(existing_ids, default=0) + 1
# check if folder starting with that id already exists, if not create a new one with next available id
if not next_id in existing_ids:    
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    output_dir = os.path.join(base_dir, f"{next_id}-{timestamp}")
    os.makedirs(output_dir, exist_ok=True)
else: 
    folder = [f for f in os.listdir(base_dir) if f.startswith(str(next_id))][0]
    output_dir = os.path.join(base_dir, folder)

print(output_dir)

for d in distances:
    command = f"/home/tommaso-peduzzi/Documents/clayg/cmake-build-debug/clayg {d} {d} {start} {end} {",".join(decoders)} {output_dir} p_step=*{step} dump=false runs=200000"
    print(f"Running: {command}")
    # Execute the command in an external shell in a new window
    subprocess.Popen(["xterm", "-e", f'cd "{cwd}" && {command}; bash'])

/home/tommaso-peduzzi/Documents/clayg/data/treshold_plots/19-2025-06-25_08-26-44
Running: /home/tommaso-peduzzi/Documents/clayg/cmake-build-debug/clayg 4 4 0.001 0.01 uf,clayg /home/tommaso-peduzzi/Documents/clayg/data/treshold_plots/19-2025-06-25_08-26-44 p_step=*1.2463361576762255 dump=false runs=200000
Running: /home/tommaso-peduzzi/Documents/clayg/cmake-build-debug/clayg 6 6 0.001 0.01 uf,clayg /home/tommaso-peduzzi/Documents/clayg/data/treshold_plots/19-2025-06-25_08-26-44 p_step=*1.2463361576762255 dump=false runs=200000


In [3]:
import glob
import re
import os
import plotly.graph_objects as go

# Updated pattern for new filenames
pattern = re.compile(r"results_(\w+)_d=(\d+)\.txt")

plot_ids = [11]
fig = go.Figure()
title = f"Threshold Plot ClAYG and Union Find\n"
title += f"(10000 trials per point)"

decoder_colors = {
    'uf': [
        "#deebf7", "#c6dbef", "#9ecae1", "#6baed6", "#4292c6",
        "#3182bd", "#1f77b4", "#2171b5", "#08519c", "#08306b",
        "#ffcccc", "#ff9999", "#ff6666", "#ff3333", "#ff0000"
    ],
    'clayg': [
        "#ff0000", "#ff7f00", "#0e550e", "#0000ff",
        "#4b0082", "#9400d3", "#8b00ff", "#ff00ff", "#ff1493",
        "#ff69b4", "#ffc0cb", "#ffa07a", "#fa8072", "#f08080"
    ]
}

smoothing_factor = 0.5

In [6]:
plot_ids = [18]

fig = go.Figure()

title = f"Threshold Plot ClAYG and Union Find\n"
title += f"(10000 trials per point)"

pattern = re.compile(r"results_(\w+)_d=(\d+)\.txt")

all_data = []

i = 0
for plot_id in plot_ids:
    data_folder = os.path.join("data/treshold_plots", f"{plot_id}*")
    matching_folders = glob.glob(data_folder)
    if not matching_folders:
        print(f"No folder found with ID {plot_id}")
        continue
    folder = matching_folders[0]
    files = glob.glob(os.path.join(folder, "results_*_d=*.txt"))
    print(files)
    for file in files:
        match = pattern.search(os.path.basename(file))
        if not match:
            continue
        decoder = match.group(1)
        depth = int(match.group(2))
        with open(file, "r") as f:
            header = f.readline().strip().split()
            # Should be: p <decoder>
            data = []
            for line in f:
                text = line.strip().split()
                if len(text) < 2:
                    continue
                p = float(text[0])
                value = float(text[1])
                if value == 0:
                    continue
                data.append((p, value))
        if not data:
            continue
        name = {"uf": "Union Find", "clayg": "ClAYG"}.get(decoder, decoder)
        print(f"Decoder: {name}, Depth: {depth}, Data points: {len(data)}")
        i += 1
        ps, vals = zip(*data)
        fig.add_trace(go.Scatter(
            x=ps, y=vals, mode="lines+markers",
            name=f"{name} d={depth}",
            line=dict(color=decoder_colors[decoder][i % len(decoder_colors[decoder])], shape="spline", smoothing=smoothing_factor, width=4),
            legendgroup=f"{name} {plot_id}",
            legendgrouptitle_text=f"{name}",
        ))
        all_data.append((decoder, depth, data))

# add a vertical line at p=0.017
fig.add_shape(
    type="line", x0=0.016, x1=0.016, y0=0, y1=0.4,
    line=dict(color=decoder_colors['clayg'][0], width=4, dash="dash"),
    legendgroup="Thresholds",
    name="Threshold ClAYG"
)
fig.add_shape(
    type="line", x0=0.0242, x1=0.0242, y0=0, y1=0.4,
    line=dict(color=decoder_colors['uf'][2], width=4    , dash="dash"),
    name="Threshold Union Find",
    legendgroup="Thresholds"
)

fig.update_layout(
    title=title,
    xaxis_title="Physical Error Rate (p)",
    yaxis_title="Logical Error Rate (l)",
    template="plotly_white",
    hoverlabel_namelength=-1,
    legend=dict(groupclick="toggleitem"),
    font=dict(size=20),
)

loglog = True
if loglog:
    fig.update_layout(xaxis_type="log", yaxis_type="log")
else:
    fig.update_layout(
        yaxis=dict(range=[0, 0.18], type="linear"),
        xaxis=dict(range=[0, 0.035], type="linear"),
    )

for decoder, depth, points in all_data:
    ps, vals = zip(*points)
    # fit function: l = a * p ** c and output c
    import numpy as np
    from scipy.optimize import curve_fit
    def fit_func(p, a, c):
        return a * p ** c
    try:
        popt, _ = curve_fit(fit_func, ps, vals)
        a, c = popt
        print(f"Fitted parameters for {decoder} d={depth} => (d+1)/2 = {(depth+1)/2}: c={c}")
    except:
        print(f"Could not fit data for {decoder} d={depth}")
        continue
    
fig.show(renderer="browser")

['data/treshold_plots/18-2025-06-24_22-43-48/results_clayg_d=4.txt', 'data/treshold_plots/18-2025-06-24_22-43-48/results_clayg_d=6.txt', 'data/treshold_plots/18-2025-06-24_22-43-48/results_uf_d=4.txt', 'data/treshold_plots/18-2025-06-24_22-43-48/results_uf_d=6.txt']
Decoder: ClAYG, Depth: 4, Data points: 8
Decoder: ClAYG, Depth: 6, Data points: 5
Decoder: Union Find, Depth: 4, Data points: 7
Decoder: Union Find, Depth: 6, Data points: 5
Fitted parameters for clayg d=4 => (d+1)/2 = 2.5: c=1.9517812445401423
Fitted parameters for clayg d=6 => (d+1)/2 = 3.5: c=2.793090478756086
Fitted parameters for uf d=4 => (d+1)/2 = 2.5: c=1.8848084351653789
Fitted parameters for uf d=6 => (d+1)/2 = 3.5: c=3.369412028815988
Opening in existing browser session.


In [ ]:
import numpy as np


plot_ids = [11]

title = "Under Threshold Plot ClAYG\n"
title += f"(10000 trials per point)"

for plot_id in plot_ids:
    data_folder = os.path.join("data/treshold_plots", f"{plot_id}-*")
    matching_folders = glob.glob(data_folder)
    if not matching_folders:
        print(f"No folder found with ID {plot_id}")
        continue

    folder_name = os.path.basename(matching_folders[0])
    data_by_probability = {}  # [decoder][p][depth] = value
    files = glob.glob(os.path.join(matching_folders[0], "d=*.txt"))
    files += glob.glob(os.path.join(matching_folders[0], "d_*.txt"))
    pattern = re.compile(r"d[=_](\d+)\.txt")

    # each file is in the format
    # d=18.txt or d_18.txt
    # and contains the data for a specific depth in the format
    # p decoder1 decoder2 decdoder3
    # we want to extract the depth from the filename
    # and the data from the file
    for file in files:
        match = pattern.search(file)
        if not match:
            continue
        depth = int(match.group(1))
        with open(file, "r") as f:
            # read the first line to get the header
            header = f.readline().strip().split()
            # extract the decoders from the first line
            decoders = header[1:]
            for line in f:
                data = {}
                # read the data from the file
                text = line.strip().split()
                # extract the p value and the decoder values
                p = float(text[0])
                values = list(map(float, text[1:]))
                # data by depth strucuture [depth][p] = dict[decoder1, decoder2, decoder3]
                for i in range(len(values)):
                    decoder = decoders[i]
                    data[decoder] = values[i]
                for decoder in decoders:
                    if not decoder in data_by_probability:
                        data_by_probability[decoder] = {}
                    if not p in data_by_probability[decoder]:
                        data_by_probability[decoder][p] = {}
                    if decoder in data:
                        data_by_probability[decoder][p][depth] = data[decoder]

fig = go.Figure()
title = f"Under Threshold Plot ClAYG\n"
title += f"(10000 trials per point)"

data = data_by_probability['clayg']
probabilities = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06]
i = 0
for p, depths in data.items():
    if not (p > 0.005 and p <= 0.016):
        continue
    color_index = i % len(decoder_colors[decoder])
    print(depths)
    sorted_depths_by_depths = dict(sorted(depths.items()))
    
    # regression over depths
    depths_array = np.array(list(sorted_depths_by_depths.keys()))
    values_array = np.array(list(sorted_depths_by_depths.values()))
    if len(depths_array) < 2:
        continue
    # fit exponential regression
    coefficients = np.polyfit(np.log(depths_array), np.log(values_array), 1)
    regression_line = np.exp(coefficients[1]) * (depths_array ** coefficients[0])
    # draw regression line
    fig.add_trace(go.Scatter(
        x=depths_array, y=regression_line, mode="lines",
        name=f"Regression p={p}",
        marker=dict(color=decoder_colors['clayg'][color_index], size=10),
        line=dict(color=decoder_colors[decoder][color_index], shape="linear", smoothing=smoothing_factor, width=3),
    ))
    print(f"p={p}, depths={sorted_depths_by_depths}")
    
    fig.add_trace(go.Scatter(
        x=list(sorted_depths_by_depths.keys()), y=list(sorted_depths_by_depths.values()), mode="lines+markers",
        name=f"p={p}",
        marker=dict(color=decoder_colors['clayg'][color_index], size=10),
    ))
    i += 1
    
fig.update_layout(xaxis_type="log", yaxis_type="log")
fig.update_layout(
    title=title,
    xaxis_title="Code Size (d)",
    yaxis_title="Logical Error Rate (l)",
    template="plotly_white",
    hoverlabel_namelength=-1,
    legend=dict(groupclick="toggleitem"),
    font=dict(size=20),
)
fig.show(renderer="browser")

NameError: name 'os' is not defined